In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen =ImageDataGenerator(rescale = 1)

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_1d304de22f0c4824ad8b8f7107066adf = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_1d304de22f0c4824ad8b8f7107066adf = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_1d304de22f0c4824ad8b8f7107066adf = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='yvQ70mVUEc95IjehSF355GfzF8Sb1omc_1pTZEza658K',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_1d304de22f0c4824ad8b8f7107066adf)

streaming_body_2 = client_1d304de22f0c4824ad8b8f7107066adf.get_object(Bucket='braintumor-donotdelete-pr-dgvw4qzb50ztqk', Key='dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
pwd

'/home/wsuser/work'

In [6]:

x_train = train_datagen.flow_from_directory('/home/wsuser/work/dataset/Brain_Tumor_Train_Test_Folders/train_set',target_size = (64,64),batch_size = 32, class_mode = 'binary')
x_test =  test_datagen.flow_from_directory('/home/wsuser/work/dataset/Brain_Tumor_Train_Test_Folders/test_set',target_size = (64,64),batch_size = 32, class_mode = 'binary')

Found 180 images belonging to 2 classes.
Found 73 images belonging to 2 classes.


In [7]:
print(x_train.class_indices)

{'no': 0, 'yes': 1}


In [8]:
model = Sequential()

In [9]:
model.add(Convolution2D(32,(3,3),input_shape = (64,64,3),activation = 'relu')) 
#no. of feature detectors, size of feature detector, input shape and activation function

In [10]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [11]:
model.add(Flatten())

In [12]:
model.add(Dense(40 ,kernel_initializer = 'uniform',activation = 'relu'))

In [13]:
model.add(Dense(1,activation = 'softmax',kernel_initializer ='uniform'))

In [14]:
model.compile(loss = 'binary_crossentropy',optimizer = "adam",metrics = ["accuracy"])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 30752)             0         
_________________________________________________________________
dense (Dense)                (None, 40)                1230120   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 41        
Total params: 1,231,057
Trainable params: 1,231,057
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit_generator(x_train, steps_per_epoch = 5,epochs = 10,validation_data = x_test,validation_steps = 40)

/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
5/5 [==============================] - 3s 536ms/step - loss: 0.6587 - accuracy: 0.6278 - val_loss: 61.2434 - val_accuracy: 0.4795
Epoch 2/10
5/5 [==============================] - 1s 224ms/step - loss: 0.5987 - accuracy: 0.6449
Epoch 3/10
5/5 [==============================] - 1s 226ms/step - loss: 0.5503 - accuracy: 0.6786
Epoch 4/10
5/5 [==============================] - 1s 228ms/step - loss: 0.5928 - accuracy: 0.6064
Epoch 5/10
5/5 [==============================] - 1s 211ms/step - loss: 0.5625 - accuracy: 0.6396
Epoch 6/10
5/5 [==============================] - 1s 219ms/step - loss: 0.5728 - accuracy: 0.6314
Epoch 7/10
5/5 [==============================] - 1s 240ms/step - loss: 0.5838 - accuracy: 0.6617
Epoch 8/10
5/5 [==============================] - 1s 223ms/step - loss: 0.5413 - accuracy: 0.6748
Epoch 9/10
5/5 [==============================] - 1s 248ms/step - loss: 0.5052 - accuracy: 0.7055
Epoch 10/10
5/5 [==============================] - 1s 234ms/step - loss: 0.

In [17]:
model.save("braintumor.h5")

In [18]:
!tar -zcvf image-classification-model_new.tgz braintumor.h5

braintumor.h5


In [19]:
ls -1

braintumor.h5
dataset/
image-classification-model_new.tgz


In [20]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 24.7 MB/s eta 0:00:01


In [21]:
from ibm_watson_machine_learning import APIClient

In [22]:
wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"RXgLtWmDdOu-CjVMGMHMZzewsg6bJHsWNgnH5QkNktXL"
}

In [23]:
client = APIClient(wml_credentials=wml_credentials)

In [24]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [25]:
space_uid=guid_from_space_name(client,'deployment_brain_t')
print("Space UID=" + space_uid)

StopIteration: 

In [27]:
client.set.default_space(space_uid)

'SUCCESS'

In [28]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [29]:
#software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7")
software_spec_uid

'65e171d7-72d1-55d9-8ebb-f813d620c9bb'

In [30]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})

model_id = client.repository.get_model_uid(model_details)

In [1]:
model_id

NameError: name 'model_id' is not defined

In [ ]:
client.repository.download(model_id,'braintumor.tar.gz')

In [ ]:
pwd